# Bitcoin Knowledge Chatbot w/ LlamaIndex
By Jon Austen

Examples:
- https://gita.kishans.in/
- https://www.chatpdf.com/
- https://gpt-index.readthedocs.io/en/latest/how_to/customization/custom_llms.html

In [2]:
!pip install llama_index
!pip install langchain
!pip install ipywidgets
!pip install openai
!pip install gpt_index

In [3]:
# change to use dotenv
import os
from dotenv import load_dotenv
# load from hidden .env file in local workspace
load_dotenv()

secret_key = os.environ['OPENAI_API_KEY']
#print(secret_key)


## PDF Example

In [4]:
from llama_index import GPTSimpleVectorIndex
from pathlib import Path
from gpt_index import download_loader

PDFReader = download_loader("PDFReader")

loader = PDFReader()
documents = loader.load_data(file=Path('./data/bitcoin.pdf'))
pdf_index = GPTSimpleVectorIndex.from_documents(documents)
pdf_index.save_to_disk('data/pdf_index.json')

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (3552 > 1024). Running this sequence through the model will result in indexing errors
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 5899 tokens


In [ ]:
response = pdf_index.query("What is Bitcoin?")
print(response)

## Wikipedia Example

In [5]:
from llama_index import download_loader
from llama_index.node_parser import SimpleNodeParser

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()
wikidocs = loader.load_data(pages=['Bitcoin'])
parser = SimpleNodeParser()

#wiki_btc_index = GPTSimpleVectorIndex.from_documents(wikidocs)
nodes = parser.get_nodes_from_documents(wikidocs)
wiki_index = GPTSimpleVectorIndex(nodes)
wiki_index.save_to_disk('data/wiki_index.json')

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 12853 tokens


In [ ]:
response = wiki_index.query("What is a Bitcoin transaction?")
print(response)

# Setup Custom LLM

In [6]:

from llama_index import LLMPredictor, GPTSimpleVectorIndex, PromptHelper, ServiceContext
from langchain import OpenAI

# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 256
# set maximum chunk overlap
max_chunk_overlap = 20

llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.1, model_name="text-davinci-003"))
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

custom_LLM_index = GPTSimpleVectorIndex(nodes, service_context=service_context)
custom_LLM_index.save_to_disk('data/custom_LLM_index.json')

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 12853 tokens


In [ ]:
response = custom_LLM_index.query("What is block height?")
print(response)

## YouTube Index

In [7]:
YoutubeTranscriptReader = download_loader("YoutubeTranscriptReader")

loader = YoutubeTranscriptReader()
yt_docs = loader.load_data(ytlinks=['https://www.youtube.com/watch?v=KeTaO-i49kw&ab_channel=TraderUniversity'])
yt_index = GPTSimpleVectorIndex.from_documents(yt_docs)
yt_index.save_to_disk('data/yt_index.json')

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 1824 tokens


In [ ]:
response = yt_index.query("What is the Byzantine Generals problem?")
print(response)

# Chatbot Class - Just include your index

In [8]:
# Create an index of your documents, this takes up to 60 seconds to load

from llama_index import GPTSimpleVectorIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader('data').load_data()
index = GPTSimpleVectorIndex.from_documents(documents)

## We dont save the master index
#index.save_to_disk('index.json')

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 447047 tokens


In [9]:
import openai
import json

class Chatbot:
    def __init__(self, api_key, index):
        self.index = index
        openai.api_key = api_key
        self.chat_history = []

    def generate_response(self, user_input):
        prompt = "\n".join([f"{message['role']}: {message['content']}" for message in self.chat_history[-5:]])
        prompt += f"\nUser: {user_input}"
        response = index.query(user_input)

        message = {"role": "assistant", "content": response.response}
        self.chat_history.append({"role": "user", "content": user_input})
        self.chat_history.append(message)
        return message
    
    def load_chat_history(self, filename):
        try:
            with open(filename, 'r') as f:
                self.chat_history = json.load(f)
        except FileNotFoundError:
            pass

    def save_chat_history(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.chat_history, f)


In [ ]:
# Swap out your index below for whatever knowledge base you want

bot = Chatbot(secret_key, index=index)
bot.load_chat_history("chat_history.json")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "goodbye"]:
        print("Bot: Goodbye!")
        bot.save_chat_history("chat_history.json")
        break
    response = bot.generate_response(user_input)
    print(f"Bot: {response['content']}")